###### Import some helper libraries 

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
from datetime import datetime , timedelta
import json
from sklearn.impute import SimpleImputer
import warnings
from geopy.geocoders import Nominatim
import time 
import googletrans
from googletrans import *

In [3]:
warnings.filterwarnings('ignore')

###### Node 1 :
* read data from first json file 

In [4]:
df = pd.read_json('tagaddod-d8ffe--MsZkGFSCtYxntenMuVF-export.json').T
df

,device_id,latitude,longitude,snapshot_datetime,meta-data,collector_id,destination_request_id
-MseZ2fAZ1SjLMaYOXy2,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:42.373120,NaN,NaN,NaN
-MseZ3tJG5FeQy7OEB6H,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:47.379586,NaN,NaN,NaN
-MseZ57uwVY_hYJ_G8nF,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:52.473970,NaN,NaN,NaN
-MseZ6JS8eUMGMr-6cF0,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:57.308858,NaN,NaN,NaN
-MseZ7XzNXS16vXvrVYO,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:07:02.335482,NaN,NaN,NaN
...,...,...,...,...,...,...,...
-Mtc0hHsnkiQaOQzOVmM,b8e1d679ea29c711,27.204013,31.174785,2022-01-17 14:33:05.208130,NaN,51,NaN
-Mtc0iVkiN2I35wNbzL9,b8e1d679ea29c711,27.204013,31.174785,2022-01-17 14:33:10.192465,NaN,51,NaN
-Mtc0k6hQNhX_wTsUoS8,b8e1d679ea29c711,27.204012,31.174785,2022-01-17 14:33:16.780540,NaN,51,NaN
-Mtc0kw8uNekqxExUPw7,b8e1d679ea29c711,27.204012,31.174785,2022-01-17 14:33:20.137822,NaN,51,NaN


###### collector location is an index make it as feature 

In [5]:
df.reset_index(inplace = True)

In [6]:
df.rename(columns = {'index' : 'collector_location'} , inplace = True)

###### check Percentage of missing records 

In [7]:
df.isnull().mean()*100

collector_location         0.000000
device_id                  0.093633
latitude                   0.093633
longitude                  0.093633
snapshot_datetime          0.093633
meta-data                 99.953184
collector_id              14.981273
destination_request_id    62.593633
dtype: float64

###### meta-data & destination request id have much missing we cant get benefit from them 

In [8]:
df.drop(['meta-data' , 'destination_request_id'] , inplace = True , axis = 1)

In [9]:
# Not understandable row !!! 
df.drop([2135] , axis = 0 , inplace = True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2135 entries, 0 to 2134
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   collector_location  2135 non-null   object
 1   device_id           2134 non-null   object
 2   latitude            2134 non-null   object
 3   longitude           2134 non-null   object
 4   snapshot_datetime   2134 non-null   object
 5   collector_id        1815 non-null   object
dtypes: object(6)
memory usage: 100.2+ KB


In [10]:
df['device_id'].unique()

array(['fe175d9955fced88', 'b8e1d679ea29c711', nan, 'N/A'], dtype=object)

###### device id has value as object (str) 'N/A' converted to nan (null)

In [11]:
df['device_id'] = df['device_id'].apply(lambda x : np.nan if x == 'N/A' else x )

In [12]:
df['device_id'].unique()

array(['fe175d9955fced88', 'b8e1d679ea29c711', nan], dtype=object)

###### data is sorted by time we can fill missing record by forward fill 

In [13]:
df['device_id'].fillna(method = 'ffill' , inplace = True)

In [14]:
df['device_id'].value_counts()

b8e1d679ea29c711    1124
fe175d9955fced88    1011
Name: device_id, dtype: int64

###### there is one collector with id = 51 

In [15]:
df['collector_id'].unique()

array([nan, '51'], dtype=object)

In [16]:
df['collector_id'].value_counts()

51    1815
Name: collector_id, dtype: int64

In [17]:
sm = SimpleImputer(strategy = 'most_frequent')

In [18]:
df['collector_id'] = sm.fit_transform(df[['collector_id']])

###### convert datatime from object (string) to datetime 

In [19]:
df['snapshot_datetime'].dtype

dtype('O')

In [11]:
df['snapshot_datetime'] = pd.to_datetime(df['snapshot_datetime'] , errors = 'coerce')

In [12]:
df['snapshot_datetime'].sort_values() 
# sort here to show strange value (Nat) -- > (not a time )

0      2022-01-05 16:06:42.373120
1      2022-01-05 16:06:47.379586
2      2022-01-05 16:06:52.473970
3      2022-01-05 16:06:57.308858
4      2022-01-05 16:07:02.335482
                  ...            
2131   2022-01-17 14:33:05.208130
2132   2022-01-17 14:33:10.192465
2133   2022-01-17 14:33:16.780540
2134   2022-01-17 14:33:20.137822
223                           NaT
Name: snapshot_datetime, Length: 2135, dtype: datetime64[ns]

###### timestamps every two seconds
we can drop this value & we can add to the previous one 2 sec 

In [22]:
df['snapshot_datetime'].loc[223] = df['snapshot_datetime'].loc[222] + timedelta(seconds=2)

In [23]:
df.isnull().mean()*100

collector_location    0.000000
device_id             0.000000
latitude              0.046838
longitude             0.046838
snapshot_datetime     0.000000
collector_id          0.000000
dtype: float64

###### latitude & longitude have some missing with low percentage we can drop them 

In [24]:
df.dropna(subset = ['latitude' , 'longitude'] , axis = 0 , inplace = True)

In [25]:
df.reset_index(inplace = True , drop = True)

In [26]:
col = ['collector_location','collector_id','device_id',
         'latitude','longitude','snapshot_datetime']

In [27]:
df = df[col]

In [28]:
df.tail(3)

,collector_location,collector_id,device_id,latitude,longitude,snapshot_datetime
2131,-Mtc0iVkiN2I35wNbzL9,51,b8e1d679ea29c711,27.204013,31.174785,2022-01-17 14:33:10.192465
2132,-Mtc0k6hQNhX_wTsUoS8,51,b8e1d679ea29c711,27.204012,31.174785,2022-01-17 14:33:16.780540
2133,-Mtc0kw8uNekqxExUPw7,51,b8e1d679ea29c711,27.204012,31.174785,2022-01-17 14:33:20.137822


# Node 2 : 
* Second json file 

In [29]:
df1 = pd.read_json('tagaddod-d8ffe--MszT9RZtTjcM5PwTCBH-export.json').T
df1

,collector_id,device_id,latitude,longitude,snapshot_datetime,destination_request_id
-Mu1Ke3YXnTlv_8zTedq,99,a5f55343f48cf3b2,30.915783,30.299763,2022-01-22 17:10:22.499088,NaN
-Mu1KfCkHPhyBE-_P9_y,99,a5f55343f48cf3b2,30.915783,30.299763,2022-01-22 17:10:27.184939,NaN
-Mu1KgR2hK2WOQDXF6d1,99,a5f55343f48cf3b2,30.915275,30.300481,2022-01-22 17:10:32.195519,NaN
-Mu1KheX3sUPsJ_26x0F,99,a5f55343f48cf3b2,30.915274,30.300482,2022-01-22 17:10:37.218805,NaN
-Mu1KisXMKQhSuocFh8c,99,a5f55343f48cf3b2,30.915278,30.300476,2022-01-22 17:10:42.210450,NaN
...,...,...,...,...,...,...
-MzvGt-m9CfNkqsIPlbr,99,3967850978244b31,31.092501,29.737423,2022-04-05 21:50:45.554494,NaN
-MzvGtefijvGFluLYhyy,99,3967850978244b31,31.092501,29.737423,2022-04-05 21:50:48.235675,NaN
-MzvGv-n48kWM9rqV-Jn,99,3967850978244b31,31.092501,29.737423,2022-04-05 21:50:53.747493,NaN
-MzvGw3gLwADyqQG61Gf,99,3967850978244b31,31.092501,29.737423,2022-04-05 21:50:58.092109,NaN


In [30]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18248 entries, -Mu1Ke3YXnTlv_8zTedq to meta-data
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   collector_id            18248 non-null  object
 1   device_id               18247 non-null  object
 2   latitude                18198 non-null  object
 3   longitude               18198 non-null  object
 4   snapshot_datetime       18247 non-null  object
 5   destination_request_id  7531 non-null   object
dtypes: object(6)
memory usage: 1.5+ MB


###### convert collector location as feature 

In [31]:
df1.reset_index(inplace = True)

In [32]:
df1.rename(columns = {'index' : 'collector_location'} , inplace = True)

In [33]:
df1.isnull().mean()*100

collector_location         0.000000
collector_id               0.000000
device_id                  0.005480
latitude                   0.274003
longitude                  0.274003
snapshot_datetime          0.005480
destination_request_id    58.729724
dtype: float64

###### Drop destination_request_id has many missing 

In [34]:
df1.drop(['destination_request_id'] , inplace = True , axis = 1)

In [35]:
df1.drop([18247] , axis = 0 , inplace = True)

In [36]:
df1.isnull().mean()*100

collector_location    0.000000
collector_id          0.000000
device_id             0.000000
latitude              0.268537
longitude             0.268537
snapshot_datetime     0.000000
dtype: float64

In [37]:
df1['collector_id'].unique()

array(['99'], dtype=object)

In [38]:
df1['device_id'].unique()

array(['a5f55343f48cf3b2', '96ddc11dcb259ebd', '3967850978244b31'],
      dtype=object)

###### convert date time from str to datetime 

In [39]:
df1['snapshot_datetime'] = pd.to_datetime(df1['snapshot_datetime'] 
                                          , errors = 'coerce' ,
                                          format = '%Y-%m-%d %H:%M:%S')

In [40]:
df1.dtypes

collector_location            object
collector_id                  object
device_id                     object
latitude                      object
longitude                     object
snapshot_datetime     datetime64[ns]
dtype: object

In [41]:
df1.dropna(subset = ['latitude' , 'longitude'] , axis = 0 , inplace = True)

In [42]:
df1.isnull().mean()*100

collector_location    0.0
collector_id          0.0
device_id             0.0
latitude              0.0
longitude             0.0
snapshot_datetime     0.0
dtype: float64

In [43]:
df1.reset_index(inplace = True , drop = True)

In [44]:
col

['collector_location',
 'collector_id',
 'device_id',
 'latitude',
 'longitude',
 'snapshot_datetime']

In [45]:
df1 = df1[col]

In [46]:
df1.tail(3)

,collector_location,collector_id,device_id,latitude,longitude,snapshot_datetime
18195,-MzvGtefijvGFluLYhyy,99,3967850978244b31,31.092501,29.737423,2022-04-05 21:50:48.235675
18196,-MzvGv-n48kWM9rqV-Jn,99,3967850978244b31,31.092501,29.737423,2022-04-05 21:50:53.747493
18197,-MzvGw3gLwADyqQG61Gf,99,3967850978244b31,31.092501,29.737423,2022-04-05 21:50:58.092109


# Node 3 :
third json file 

In [47]:
df2 = pd.read_json('tagaddod-d8ffe--MwWeqpG3yuQD7G2wcp8-export.json').T
df2

,collector_id,device_id,latitude,longitude,snapshot_datetime,destination_request_id
-MwWgmIKz8ZLI-S3HtAp,157,0891846aad4e869e,29.952633,30.888284,2022-02-22 15:51:02.869005,NaN
-MwWgmyb-JvddB_4D41K,157,0891846aad4e869e,29.952633,30.888284,2022-02-22 15:51:05.639681,NaN
-MwWgoX1YBF7jLDYPzHj,157,0891846aad4e869e,29.952638,30.888283,2022-02-22 15:51:12.002744,NaN
-MwWgpLvpj_9XZK1rza7,157,0891846aad4e869e,29.952638,30.888283,2022-02-22 15:51:15.387694,NaN
-MwWgqwJCVqXiZnRyYxK,157,0891846aad4e869e,29.952642,30.888282,2022-02-22 15:51:21.876488,NaN
...,...,...,...,...,...,...
-N3ab30MLr8vzHO5SEFr,157,0891846aad4e869e,30.566616,31.012179,2022-06-03 00:49:43.255626,NaN
-N3ab4_XRd5Oua4TjIhe,157,0891846aad4e869e,30.566256,31.012319,2022-06-03 00:49:49.666424,NaN
-N3ab5QXpG-qgP55DQTf,157,0891846aad4e869e,30.566256,31.012319,2022-06-03 00:49:53.122114,NaN
-N3ab72RE93xSqsFMpMq,157,0891846aad4e869e,30.565279,31.012417,2022-06-03 00:49:59.771898,NaN


###### convert collector location as feature 

In [48]:
df2.reset_index(inplace = True)

In [49]:
df2.rename(columns = {'index' : 'collector_location'} , inplace = True)

In [50]:
df2.isnull().mean()*100

collector_location         0.000000
collector_id               0.000000
device_id                  0.058173
latitude                   0.814427
longitude                  0.814427
snapshot_datetime          0.058173
destination_request_id    26.934264
dtype: float64

###### Drop destination_request_id has many missing 

In [51]:
df2.drop(['destination_request_id'] , axis = 1 , inplace = True)

In [52]:
df2.drop([1718] , axis = 0 , inplace = True)

In [53]:
df2.isnull().mean()*100

collector_location    0.000000
collector_id          0.000000
device_id             0.000000
latitude              0.756694
longitude             0.756694
snapshot_datetime     0.000000
dtype: float64

###### convert date time from str to datetime 

In [54]:
df2['snapshot_datetime'] = pd.to_datetime(df2['snapshot_datetime'] , errors = 'coerce')

In [55]:
df2.isnull().mean()*100

collector_location    0.000000
collector_id          0.000000
device_id             0.000000
latitude              0.756694
longitude             0.756694
snapshot_datetime     0.000000
dtype: float64

In [56]:
df2.dropna(subset = ['latitude','longitude'] , inplace = True , axis = 0)

In [57]:
df2.reset_index(inplace = True , drop = True)

In [58]:
df2 = df2[col]

In [59]:
df2.tail(3)

,collector_location,collector_id,device_id,latitude,longitude,snapshot_datetime
1702,-N3ab4_XRd5Oua4TjIhe,157,0891846aad4e869e,30.566256,31.012319,2022-06-03 00:49:49.666424
1703,-N3ab5QXpG-qgP55DQTf,157,0891846aad4e869e,30.566256,31.012319,2022-06-03 00:49:53.122114
1704,-N3ab72RE93xSqsFMpMq,157,0891846aad4e869e,30.565279,31.012417,2022-06-03 00:49:59.771898


# Node 4 : 
fourth json file 

In [60]:
df3 = pd.read_json('tagaddod-d8ffe--N4SUsENXI1OF1qf8VxT-export.json').T
df3

,collector_id,device_id,latitude,longitude,snapshot_datetime
-N4SWCnP5uAPMk2vlgsh,257,a685f0f069042727,30.562314,31.560081,2022-06-13 16:43:17.338230
-N4SWDqDqUVYxPTGsEYr,257,a685f0f069042727,30.562314,31.560081,2022-06-13 16:43:21.614333
-N4SWFG074ZdMJm5B7h3,257,a685f0f069042727,30.562314,31.560081,2022-06-13 16:43:27.425884
-N4SWG6b3kT3WQxIjw_x,257,a685f0f069042727,30.562314,31.560081,2022-06-13 16:43:30.918961
-N4SWHi3Zq_pQl99SaVN,257,a685f0f069042727,30.562314,31.560081,2022-06-13 16:43:37.476021
...,...,...,...,...,...
-N4SsdlW6fgRvBC5U-2x,257,a685f0f069042727,30.562312,31.56008,2022-06-13 18:25:41.217370
-N4SsepalJNpxDsjo3xf,257,a685f0f069042727,30.562305,31.560091,2022-06-13 18:25:45.574352
-N4SsgNE5GU2N4_W3rL6,257,a685f0f069042727,30.562304,31.560093,2022-06-13 18:25:51.887048
-N4SshUlcPbVssLDwcRl,257,a685f0f069042727,30.562304,31.560093,2022-06-13 18:25:56.465310


In [61]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, -N4SWCnP5uAPMk2vlgsh to meta-data
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   collector_id       84 non-null     object
 1   device_id          83 non-null     object
 2   latitude           83 non-null     object
 3   longitude          83 non-null     object
 4   snapshot_datetime  83 non-null     object
dtypes: object(5)
memory usage: 6.0+ KB


In [62]:
df3.reset_index(inplace = True)

In [63]:
df3.rename(columns = {'index' : 'collector_location'} , inplace = True)

In [64]:
df3.drop([83] , axis = 0 , inplace = True)

In [65]:
df3.reset_index(inplace = True , drop = True)

In [66]:
df3.isnull().mean()*100

collector_location    0.0
collector_id          0.0
device_id             0.0
latitude              0.0
longitude             0.0
snapshot_datetime     0.0
dtype: float64

In [67]:
df3['snapshot_datetime'] = pd.to_datetime(df3['snapshot_datetime'] , errors = 'coerce')

In [68]:
df3 = df3[col]

In [69]:
df3.tail(3)

,collector_location,collector_id,device_id,latitude,longitude,snapshot_datetime
80,-N4SsepalJNpxDsjo3xf,257,a685f0f069042727,30.562305,31.560091,2022-06-13 18:25:45.574352
81,-N4SsgNE5GU2N4_W3rL6,257,a685f0f069042727,30.562304,31.560093,2022-06-13 18:25:51.887048
82,-N4SshUlcPbVssLDwcRl,257,a685f0f069042727,30.562304,31.560093,2022-06-13 18:25:56.465310


### Know concate our four nodes and make some feature engineering 

In [70]:
data = pd.concat([df , df1 , df2 , df3] , axis = 0,ignore_index = True)

In [71]:
data

,collector_location,collector_id,device_id,latitude,longitude,snapshot_datetime
0,-MseZ2fAZ1SjLMaYOXy2,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:42.373120
1,-MseZ3tJG5FeQy7OEB6H,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:47.379586
2,-MseZ57uwVY_hYJ_G8nF,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:52.473970
3,-MseZ6JS8eUMGMr-6cF0,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:57.308858
4,-MseZ7XzNXS16vXvrVYO,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:07:02.335482
...,...,...,...,...,...,...
22115,-N4SsdHjs4Ks-F1BnzNV,257,a685f0f069042727,30.562312,31.56008,2022-06-13 18:25:39.247044
22116,-N4SsdlW6fgRvBC5U-2x,257,a685f0f069042727,30.562312,31.56008,2022-06-13 18:25:41.217370
22117,-N4SsepalJNpxDsjo3xf,257,a685f0f069042727,30.562305,31.560091,2022-06-13 18:25:45.574352
22118,-N4SsgNE5GU2N4_W3rL6,257,a685f0f069042727,30.562304,31.560093,2022-06-13 18:25:51.887048


In [72]:
data.isnull().mean()*100

collector_location    0.0
collector_id          0.0
device_id             0.0
latitude              0.0
longitude             0.0
snapshot_datetime     0.0
dtype: float64

###### Extract some valuable info from date & time 

In [73]:
data['Year'] = data['snapshot_datetime'].dt.year

In [74]:
data['Month'] = data['snapshot_datetime'].dt.month_name()

In [75]:
data['Day'] = data['snapshot_datetime'].dt.day_name()

In [76]:
data['Hour'] = data['snapshot_datetime'].dt.hour

## We can get DayPeriod & Season : 

In [77]:
def map_hours(x):
    if x in range(0, 13):
        return 'morning'
    elif x in range(13, 19):
        return 'afternoon'
    else:
        return 'evening'
    
    
def map_months(x):
    if x in ['December', 'January', 'February']:
        return 'Winter'
    elif x in ['March', 'April', 'May']:
        return 'Spring'
    elif x in ['June', 'July', 'August']:
        return 'Summer'
    elif x in ['September', 'October', 'November']:
        return 'Autumn'

In [78]:
data['DayPeriod'] = data['Hour'].apply(map_hours)

In [79]:
data['Season'] = data['Month'].apply(map_months)

In [80]:
data

,collector_location,collector_id,device_id,latitude,longitude,snapshot_datetime,Year,Month,Day,Hour,DayPeriod,Season
0,-MseZ2fAZ1SjLMaYOXy2,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:42.373120,2022,January,Wednesday,16,afternoon,Winter
1,-MseZ3tJG5FeQy7OEB6H,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:47.379586,2022,January,Wednesday,16,afternoon,Winter
2,-MseZ57uwVY_hYJ_G8nF,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:52.473970,2022,January,Wednesday,16,afternoon,Winter
3,-MseZ6JS8eUMGMr-6cF0,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:06:57.308858,2022,January,Wednesday,16,afternoon,Winter
4,-MseZ7XzNXS16vXvrVYO,51,fe175d9955fced88,30.042313,31.33642,2022-01-05 16:07:02.335482,2022,January,Wednesday,16,afternoon,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...
22115,-N4SsdHjs4Ks-F1BnzNV,257,a685f0f069042727,30.562312,31.56008,2022-06-13 18:25:39.247044,2022,June,Monday,18,afternoon,Summer
22116,-N4SsdlW6fgRvBC5U-2x,257,a685f0f069042727,30.562312,31.56008,2022-06-13 18:25:41.217370,2022,June,Monday,18,afternoon,Summer
22117,-N4SsepalJNpxDsjo3xf,257,a685f0f069042727,30.562305,31.560091,2022-06-13 18:25:45.574352,2022,June,Monday,18,afternoon,Summer
22118,-N4SsgNE5GU2N4_W3rL6,257,a685f0f069042727,30.562304,31.560093,2022-06-13 18:25:51.887048,2022,June,Monday,18,afternoon,Summer


## Lat & Long :

In [81]:
geolocator = Nominatim(user_agent="Task")

In [82]:
geolocator.reverse('30.562304,31.560093').raw['address']

{'road': 'شارع الطاهره',
 'suburb': 'طاهرة',
 'city': 'الزقازيق',
 'state': 'الشرقية',
 'ISO3166-2-lvl4': 'EG-SHR',
 'postcode': '44862',
 'country': 'مصر',
 'country_code': 'eg'}

In [83]:
def GetAddress(r):
    try:
        geolocator = Nominatim(user_agent="Task".format(np.random.randint(50,1000000)))
        address = geolocator.reverse(r).raw['address']
        return address
    except:
        return np.nan

    
GetAddress('30.916706,29.6046081')

{'neighbourhood': 'بهيج',
 'village': 'قرية بهيج',
 'state': 'الإسكندرية',
 'ISO3166-2-lvl4': 'EG-ALX',
 'postcode': '23719',
 'country': 'مصر',
 'country_code': 'eg'}

In [84]:
data['latlong'] = data['latitude'].astype('str')+','+data['longitude'].astype('str')

In [85]:
# take a long time to run :(
#data['address'] = data['latlong'].apply(GetAddress)
#data.to_csv('CleanedData.csv' , index = False)

In [86]:
data = pd.read_csv('CleanedData.csv' , encoding = 'utf-8')

In [87]:
data.dropna(subset = 'address' , axis = 0 , inplace = True)

In [88]:
data.reset_index(inplace = True , drop = True)

###### when reading our data from csv file address as string so convert to dict to deal with it 

In [89]:
data['address'] = data['address'].apply(lambda x : eval(x))

In [90]:
data['address'].loc[0]

{'house_number': '67',
 'road': 'شارع الشيخ طه الديناري',
 'neighbourhood': 'منطقة 7',
 'suburb': 'مدينة نصر',
 'city': 'القاهرة',
 'state': 'القاهرة',
 'ISO3166-2-lvl4': 'EG-C',
 'postcode': '11765',
 'country': 'مصر',
 'country_code': 'eg'}

In [91]:
# needed info from address 
lst = ['house_number' , 'road' , 'town' , 'suburb' , 'state' , 'city']

In [92]:
data['address'].loc[0].get('road')

'شارع الشيخ طه الديناري'

In [93]:
data['road'] = data['address'].apply(lambda x : x.get('road'))

In [94]:
data['state'] = data['address'].apply(lambda x : x.get('state'))

In [95]:
data['city'] = data['address'].apply(lambda x : x.get('city'))

In [96]:
data.fillna(axis = 0 , method = 'ffill' , inplace = True)

In [97]:
data['road'].unique().tolist()

['شارع الشيخ طه الديناري',
 'حارة عشرى',
 'شارع القصر العيني',
 'شارع محمد محمود',
 'موقف سيارات عمر مكرم',
 'حارة الامير قدادار',
 'شارع بستان بن قريش',
 'شارع الفلكي',
 'شارع منصور',
 'شارع التحرير',
 'شارع محمد صدقي باشا',
 'شارع عبد السلام عارف',
 'شارع منشيه المهراني',
 'شارع مصطفي ابو هيف',
 'شارع الامير قدادار',
 'شارع طلعت حرب',
 'شارع يوسف الجندي',
 'شارع الدكتور عبد الشافي محمد',
 'شارع الشيخ ريحان',
 'شارع الخطبة',
 'شارع ربيع جاد الله ابو شقاف',
 'شارع 15',
 'شارع الجمهوريه',
 'طريق ابو المطامير, دمنهور',
 'شارع حضانة درا الجنة',
 'طريق القاهرة, الاسكندرية الزراعى',
 'شارع عبد الرحمن الرافعى',
 'شارع قنال المحمودية',
 'شارع احمد العبانى',
 'شارع أطلس',
 'شارع محمد عثمان',
 'شارع محمد انور السادات',
 'شارع 14',
 'شارع 30',
 'شارع مصطفى كامل',
 'شارع الصرف الصحى',
 'شارع عزبة المطر',
 'شارع سليمان عبد الجواد',
 'شارع الفلكى للموبيليا',
 'شارع بطل السلام',
 'شارع مسجد الرحمه',
 'شارع الشهاوى',
 'شارع النصر',
 'شارع محمد عباس',
 'شارع الترعه',
 'شارع 14 مايو',
 'شارع عمر بن الخ

In [98]:
# convert some wrong values to nan
data['road'].loc[data[data['road'] == '포은대로'].index] = np.nan

In [99]:
data['city'].unique()

array(['القاهرة', 'أسيوط', 'الإسكندرية', 'دمنهور', '용인시',
       'مدينة السادس من أكتوبر', 'شبرا الخيمة', 'مدينة الشيخ زايد',
       'شبين الكوم', 'الزقازيق'], dtype=object)

In [100]:
data['city'].loc[data[data['city'] == '용인시'].index] = np.nan

In [101]:
data['state'].unique()

array(['القاهرة', 'أسيوط', 'البحيرة', 'الإسكندرية', 'مطروح', 'الجيزة',
       'القليوبية', 'المنوفية', 'الشرقية'], dtype=object)

In [102]:
data.isnull().sum()

collector_location     0
collector_id           0
device_id              0
latitude               0
longitude              0
snapshot_datetime      0
Year                   0
Month                  0
Day                    0
Hour                   0
DayPeriod              0
Season                 0
latlong                0
address                0
road                  54
state                  0
city                  97
dtype: int64

In [103]:
data.fillna(method = 'ffill' , inplace = True , axis = 0)

In [104]:
data.drop(['address' , 'latlong'] , inplace = True , axis = 1)

In [105]:
import googletrans
from googletrans import *

###### finally translate into English
* from 3 feature we get unique values and translate it & save it in dictionary to map it on whole data  
* translator take a long time for translate and has limit number of words to translate 

In [119]:
translator = googletrans.Translator()

In [109]:
road_trans = {i : translator.translate(i).text for i in data['road'].unique()[:200]}

In [110]:
di2 = {i : translator.translate(i).text for i in data['road'].unique()[200:400]}

In [112]:
di3 = {i : translator.translate(i).text for i in data['road'].unique()[400:]}

In [113]:
road_trans.update(di2)

In [114]:
road_trans.update(di3)

In [115]:
data['road'] = data['road'].map(road_trans)

In [116]:
city_trans = {i : translator.translate(i).text for i in data['city'].unique()}

In [117]:
data['city'] = data['city'].map(city_trans)

In [120]:
state_trans = {i : translator.translate(i).text for i in data['state'].unique()}

In [121]:
data['state'] = data['state'].map(state_trans)

In [122]:
data

,collector_location,collector_id,device_id,latitude,longitude,snapshot_datetime,Year,Month,Day,Hour,DayPeriod,Season,road,state,city
0,-MseZ2fAZ1SjLMaYOXy2,51,fe175d9955fced88,30.042313,31.336420,2022-01-05 16:06:42.373120,2022,January,Wednesday,16,afternoon,Winter,Sheikh Taha Al -Dinari Street,Cairo,Cairo
1,-MseZ3tJG5FeQy7OEB6H,51,fe175d9955fced88,30.042313,31.336420,2022-01-05 16:06:47.379586,2022,January,Wednesday,16,afternoon,Winter,Sheikh Taha Al -Dinari Street,Cairo,Cairo
2,-MseZ57uwVY_hYJ_G8nF,51,fe175d9955fced88,30.042313,31.336420,2022-01-05 16:06:52.473970,2022,January,Wednesday,16,afternoon,Winter,Sheikh Taha Al -Dinari Street,Cairo,Cairo
3,-MseZ6JS8eUMGMr-6cF0,51,fe175d9955fced88,30.042313,31.336420,2022-01-05 16:06:57.308858,2022,January,Wednesday,16,afternoon,Winter,Sheikh Taha Al -Dinari Street,Cairo,Cairo
4,-MseZ7XzNXS16vXvrVYO,51,fe175d9955fced88,30.042313,31.336420,2022-01-05 16:07:02.335482,2022,January,Wednesday,16,afternoon,Winter,Sheikh Taha Al -Dinari Street,Cairo,Cairo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21949,-N4SsdHjs4Ks-F1BnzNV,257,a685f0f069042727,30.562312,31.560080,2022-06-13 18:25:39.247044,2022,June,Monday,18,afternoon,Summer,Taher Street,Eastern,Zagazig
21950,-N4SsdlW6fgRvBC5U-2x,257,a685f0f069042727,30.562312,31.560080,2022-06-13 18:25:41.217370,2022,June,Monday,18,afternoon,Summer,Taher Street,Eastern,Zagazig
21951,-N4SsepalJNpxDsjo3xf,257,a685f0f069042727,30.562305,31.560091,2022-06-13 18:25:45.574352,2022,June,Monday,18,afternoon,Summer,Taher Street,Eastern,Zagazig
21952,-N4SsgNE5GU2N4_W3rL6,257,a685f0f069042727,30.562304,31.560093,2022-06-13 18:25:51.887048,2022,June,Monday,18,afternoon,Summer,Taher Street,Eastern,Zagazig


In [123]:
#data.to_csv('GCM.csv' , index = False)